In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import time

def find_4_points(frame):
    #find rightmost point 

    r_point_x=frame.shape[1]-1
    r_point_y=frame.shape[0]-1

    while frame[r_point_y,r_point_x]!= 255:
        r_point_y+=-1
        if r_point_y==0:
            r_point_y=frame.shape[0]-1
            r_point_x+=-1


    #find leftmost point 

    l_point_x=0
    l_point_y=frame.shape[0]-1

    while frame[l_point_y,l_point_x]!= 255:
        l_point_y+=-1
        if l_point_y==0:
            l_point_y=frame.shape[0]-1
            l_point_x+=+1

    #find upper point

    u_point_x=0
    u_point_y=0

    while frame[u_point_y,u_point_x]!= 255:
        u_point_x+=+1
        if u_point_x==(frame.shape[1]-1):
            u_point_x=0
            u_point_y+=+1

    #find lowest point

    d_point_x=0
    d_point_y=frame.shape[0]-1

    while frame[d_point_y,d_point_x]!= 255:
        d_point_x+=+1
        if d_point_x==(frame.shape[1]-1):
            d_point_x=0
            d_point_y+=-1
    return (u_point_y,d_point_y, l_point_x,r_point_x)



In [2]:
cap = cv2.VideoCapture(r"C:\Users\con74781\Desktop\Digital Projects\Aeiroos\Camera project\VID-20200812-WA0004.mp4") 
if (cap.isOpened()== False):  
    print("Error opening video  file") 

fourcc = cv2.VideoWriter_fourcc(*'XVID')    
out = cv2.VideoWriter('Fault_detection_2fps.avi',fourcc,3, (350,250)) # ensure frame matches output frame

template=cv2.imread(r"C:\Users\con74781\Desktop\Digital Projects\Aeiroos\Camera project\production\template_pic\good_template.jpg")



template=cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template=cv2.resize(template,(400,300))


while(cap.isOpened()): 
    ret, frame = cap.read()
    frame=cv2.resize(frame,(400,300))
    frame_to_concatenate=frame
    

    gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Template', gray_frame)
 

    if ret==False:
        break

    cnt=0
    
    # find and show the frames only matching the template 
    sift = cv2.xfeatures2d.SIFT_create()
    kp1,des1 = sift.detectAndCompute(template,None)
    kp2,des2 = sift.detectAndCompute(gray_frame,None)
    bf=cv2.BFMatcher()
    matches=bf.knnMatch(des1,des2,k=2)

    good=[]
    for match1,match2 in matches:
        if match1.distance <0.8*match2.distance:
            good.append([match1])
    sift_matches=cv2.drawMatchesKnn(gray_frame,kp1,template,kp2,good,None,flags=2)
#     frame = cv2.drawKeypoints(frame,kp1,frame)  #hide
#     cv2.imshow('Detection',frame)
   
    
    if len(good)>120:
        gray_frame = cv2.blur(gray_frame,(5,5))
        gray_frame = cv2.Canny(gray_frame,100,200)
        u,d,l,r= find_4_points(gray_frame)
        frame=frame[u:d,l:r]       
        frame=cv2.resize(frame,(400,300))

      
    #  filter for the product labels, white colour labelling 
        light_color=(250,250,250) 
        dark_color=(180,180,180)
        frame_mask= cv2.inRange(frame, dark_color,light_color)

    #  find the contours of filtered labels 
        
        contours, hierarchy = cv2.findContours(frame_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in range(0,len(contours)):
            if cv2.contourArea(contours[i])>5000:
                frame=cv2.drawContours(frame, contours, i, 255, 1)  
                rect1 = cv2.minAreaRect(contours[i])
                box1 = cv2.boxPoints(rect1)
                box1 = np.int0(box1)
                cv2.drawContours(frame,[box1],0,(0,0,255),2)
                x,y=rect1[0]
                x=int(x)
                y=int(y)
                if (x<=80 and y<=80) or (x<=110 and (190<=y<=220)): 
                    cnt+=1
        if cnt==2:
            cv2.putText(frame,"GOOD",(200,45), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0),2,cv2.LINE_8)
        else:
            cv2.putText(frame,"FAULT",(200,45), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2,cv2.LINE_8)
        frame=frame[:250,:350]        
        out.write(frame)
       
        
#         horizontal_concat = np.concatenate((frame,frame_to_concatenate), axis=1)
        cv2.imshow('Detection', frame)
        
#         print(cnt)
         
        
    cnt+=1
    if cv2.waitKey(80) & 0xFF == ord('q'): 
        break

    

cap.release() 
out.release()
cv2.destroyAllWindows() 